# Face Recognition

**Step-1 importing libraries**

In [1]:
import glob
import dlib
import cv2
import pickle
import random
import facevec
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def adjust_gamma(input_image, gamma=1.0):
    table = np.array([((iteration / 255.0) ** (1.0 / gamma)) * 255
                      for iteration in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(input_image, table)

In [3]:
def read_image(path, gamma=0.75):
    output = cv2.imread(path)
    return adjust_gamma(output, gamma=gamma)

In [4]:
def face_vector(input_image):
    faces = facevec.detector(input_image, 1)
    if not faces:
        return None

    f = faces[0]
    shape = facevec.predictor(input_image, f)
    face_descriptor = facevec.face_model.compute_face_descriptor(input_image, shape)
    return face_descriptor

In [5]:
male = 0
female = 1

In [6]:
print("Retrieving Male images ...")
sub1 = glob.glob("./data/face_data/male/*.png")
print("Retrieved {} faces !".format(len(sub1)))

print("Retrieving female images ...")
sub2 = glob.glob("./data/face_data/female/*.png")
print("Retrieved {} faces !".format(len(sub2)))


Retrieving Male images ...
Retrieved 76 faces !
Retrieving female images ...
Retrieved 64 faces !


In [8]:
vectors = dlib.vectors()
labels = dlib.array()

In [9]:
print("Reading Males images ...")
for i, sub in enumerate(sub1):
    print("Reading {} of {}\r".format(i, len(sub1)))
    face_vectors = face_vector(read_image(sub))
    if face_vectors is None:
        continue
    vectors.append(dlib.vector(face_vectors))
    labels.append(male)

Reading Males images ...
Reading 0 of 76
Reading 1 of 76
Reading 2 of 76
Reading 3 of 76
Reading 4 of 76
Reading 5 of 76
Reading 6 of 76
Reading 7 of 76
Reading 8 of 76
Reading 9 of 76
Reading 10 of 76
Reading 11 of 76
Reading 12 of 76
Reading 13 of 76
Reading 14 of 76
Reading 15 of 76
Reading 16 of 76
Reading 17 of 76
Reading 18 of 76
Reading 19 of 76
Reading 20 of 76
Reading 21 of 76
Reading 22 of 76
Reading 23 of 76
Reading 24 of 76
Reading 25 of 76
Reading 26 of 76
Reading 27 of 76
Reading 28 of 76
Reading 29 of 76
Reading 30 of 76
Reading 31 of 76
Reading 32 of 76
Reading 33 of 76
Reading 34 of 76
Reading 35 of 76
Reading 36 of 76
Reading 37 of 76
Reading 38 of 76
Reading 39 of 76
Reading 40 of 76
Reading 41 of 76
Reading 42 of 76
Reading 43 of 76
Reading 44 of 76
Reading 45 of 76
Reading 46 of 76
Reading 47 of 76
Reading 48 of 76
Reading 49 of 76
Reading 50 of 76
Reading 51 of 76
Reading 52 of 76
Reading 53 of 76
Reading 54 of 76
Reading 55 of 76
Reading 56 of 76
Reading 57 of 76

In [10]:
print("Reading Female  images ...")
for i, sub in enumerate(sub2):
    print("Reading {} of {}\r".format(i, len(sub2)))
    face_vectors = face_vector(read_image(sub))
    if face_vectors is None:
        continue
    vectors.append(dlib.vector(face_vectors))
    labels.append(female)

Reading Female  images ...
Reading 0 of 64
Reading 1 of 64
Reading 2 of 64
Reading 3 of 64
Reading 4 of 64
Reading 5 of 64
Reading 6 of 64
Reading 7 of 64
Reading 8 of 64
Reading 9 of 64
Reading 10 of 64
Reading 11 of 64
Reading 12 of 64
Reading 13 of 64
Reading 14 of 64
Reading 15 of 64
Reading 16 of 64
Reading 17 of 64
Reading 18 of 64
Reading 19 of 64
Reading 20 of 64
Reading 21 of 64
Reading 22 of 64
Reading 23 of 64
Reading 24 of 64
Reading 25 of 64
Reading 26 of 64
Reading 27 of 64
Reading 28 of 64
Reading 29 of 64
Reading 30 of 64
Reading 31 of 64
Reading 32 of 64
Reading 33 of 64
Reading 34 of 64
Reading 35 of 64
Reading 36 of 64
Reading 37 of 64
Reading 38 of 64
Reading 39 of 64
Reading 40 of 64
Reading 41 of 64
Reading 42 of 64
Reading 43 of 64
Reading 44 of 64
Reading 45 of 64
Reading 46 of 64
Reading 47 of 64
Reading 48 of 64
Reading 49 of 64
Reading 50 of 64
Reading 51 of 64
Reading 52 of 64
Reading 53 of 64
Reading 54 of 64
Reading 55 of 64
Reading 56 of 64
Reading 57 of 

# Training

In [11]:
type(vectors)

dlib.vectors

In [12]:
vec = np.array(vectors)

In [13]:
lab = np.array(labels)

> **Saving into csv file**

In [14]:
X = vec
y = lab.reshape(-1,1)

In [15]:
dataset = np.append(X,y,axis = 1)

In [16]:
np.savetxt('face_vectors.csv',dataset,delimiter = ',')

**Split dataset into traning and testing**

In [17]:
from sklearn.cross_validation import train_test_split

In [18]:
x_train, x_test, y_train, y_test = train_test_split(X,y,
                                                    test_size = 0.2)

**Building Machine Leanring Model**

In [19]:
from sklearn.svm import SVC

In [25]:
model = SVC(C = 10.0,probability=True)
model.fit(x_train,y_train) # traning
y_pred = model.predict(x_test)

C:\Users\srikanth\Anaconda\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


**Evaluation**

In [26]:
from sklearn.metrics import classification_report

In [27]:
cr = classification_report(y_test,y_pred)
print(cr)

             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00        11
        1.0       1.00      1.00      1.00        16

avg / total       1.00      1.00      1.00        27



In [28]:
from sklearn.externals import joblib # saving machine learning

In [29]:
joblib.dump(model,'gender_class.pkl')

['gender_class.pkl']